In [2]:
pip install kagglehub

In [1]:
# --- NYT articles+comments join & filter (inline version) ---
import os, re, glob
from typing import List, Tuple, Optional
import pandas as pd

def _clean_url(u):
    if pd.isna(u): return u
    u = str(u).strip()
    return u.split('?')[0].rstrip('/').lower()

def _parse_keywords(val):
    if pd.isna(val): return []
    if isinstance(val, list): return [str(x).strip() for x in val if str(x).strip()]
    return [p.strip() for p in re.split(r'[;,]\s*', str(val)) if p.strip()]

ARTICLE_COLS = ["article_id","url","pub_date","section","subsection","headline","abstract","news_desk","type_of_material","keywords"]
COMMENT_COLS = ["comment_id","parent_id","user","comment","recommendations","reply_count","editors_selection","date","article_url","article_id"]

def normalize_articles(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    cand = {
        "article_id":["articleID","asset_id","assetId","id","article_id"],
        "url":["articleURL","url","web_url","link"],
        "pub_date":["pub_date","pubDate","date","published_date","PublicationDate"],
        "section":["section","section_name","newsSection","news_desk"],
        "subsection":["subsection","subsection_name","sub_section"],
        "headline":["headline","title","main_headline","headline.main","Title"],
        "abstract":["abstract","snippet","abstract_text","lead_paragraph","Summary"],
        "news_desk":["news_desk","desk"],
        "type_of_material":["type_of_material","type","material_type_facet"],
        "keywords":["keywords","descriptors","subject","keywords_list"],
    }
    for canon, opts in cand.items():
        for o in opts:
            if o in df.columns:
                df[canon] = df[o]; break
    if "url" in df: df["url"] = df["url"].map(_clean_url)
    if "keywords" in df: df["keywords"] = df["keywords"].map(_parse_keywords)
    else: df["keywords"] = [[] for _ in range(len(df))]
    for c in ARTICLE_COLS:
        if c not in df: df[c] = pd.NA
    return df[ARTICLE_COLS]

def normalize_comments(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    cand = {
        "comment_id":["commentID","comment_id","id"],
        "parent_id":["parentID","parent_id","inReplyTo"],
        "user":["userDisplayName","user","display_name","author"],
        "comment":["commentBody","comment","text","body"],
        "recommendations":["recommendations","recommendCount","recommendedCount"],
        "reply_count":["replyCount","replies","numReplies"],
        "editors_selection":["editorsSelection","editors_selection"],
        "date":["createDate","date","timestamp"],
        "article_url":["articleURL","url","articleUrl","web_url"],
        "article_id":["articleID","asset_id","assetId","article_id"],
    }
    for canon, opts in cand.items():
        for o in opts:
            if o in df.columns:
                df[canon] = df[o]; break
    if "article_url" in df: df["article_url"] = df["article_url"].map(_clean_url)
    for c in COMMENT_COLS:
        if c not in df: df[c] = pd.NA
    return df[COMMENT_COLS]

def load_2017_2018(base_dir: str):
    a_paths = sorted(glob.glob(os.path.join(base_dir, "Articles*.csv")))
    c_paths = sorted(glob.glob(os.path.join(base_dir, "Comments*.csv")))
    if not a_paths: raise FileNotFoundError("No Articles*.csv in " + base_dir)
    if not c_paths: raise FileNotFoundError("No Comments*.csv in " + base_dir)
    arts = pd.concat([pd.read_csv(p, low_memory=False) for p in a_paths], ignore_index=True)
    comms = pd.concat([pd.read_csv(p, low_memory=False) for p in c_paths], ignore_index=True)
    return normalize_articles(arts), normalize_comments(comms)

def load_2020(base_dir: str):
    a_paths = sorted(set(glob.glob(os.path.join(base_dir,"*articles*.csv")) + glob.glob(os.path.join(base_dir,"articles*.csv"))))
    c_paths = sorted(set(glob.glob(os.path.join(base_dir,"*comments*.csv")) + glob.glob(os.path.join(base_dir,"comments*.csv"))))
    if not a_paths: raise FileNotFoundError("No *articles*.csv in " + base_dir)
    if not c_paths: raise FileNotFoundError("No *comments*.csv in " + base_dir)
    arts = pd.concat([pd.read_csv(p, low_memory=False) for p in a_paths], ignore_index=True)
    comms = pd.concat([pd.read_csv(p, low_memory=False) for p in c_paths], ignore_index=True)
    return normalize_articles(arts), normalize_comments(comms)

def join_articles_comments(articles: pd.DataFrame, comments: pd.DataFrame) -> pd.DataFrame:
    a, c = articles.copy(), comments.copy()
    can_id = a["article_id"].notna().any() and c["article_id"].notna().any()
    if can_id:
        m = c.merge(a, how="left", on="article_id", suffixes=("_c","_a"))
        missing = m["headline"].isna()
        if missing.any():
            fill = c.loc[missing, ["article_url"]].merge(
                a[["url","headline","abstract","pub_date","section","subsection","news_desk","type_of_material","keywords","article_id"]],
                how="left", left_on="article_url", right_on="url"
            )
            m.loc[missing, ["headline","abstract","pub_date","section","subsection","news_desk","type_of_material","keywords","article_id"]] = \
                fill[["headline","abstract","pub_date","section","subsection","news_desk","type_of_material","keywords","article_id"]].values
        return m
    return c.merge(a, how="left", left_on="article_url", right_on="url", suffixes=("_c","_a"))

SECTION_ALLOW = {"u.s.","us","us news","politics","business","economy"}
KEYWORD_PATTERNS = [
    r"\bpolitic(s|al)?\b", r"\bcongress\b", r"\bsenate\b", r"\bhouse of representatives\b",
    r"\bwhite house\b", r"\bpresident\b", r"\bgovern(or|ment)\b", r"\bgubernatorial\b",
    r"\belection(s)?\b", r"\bprimary\b", r"\bmidterm(s)?\b", r"\bballot\b", r"\bcampaign\b",
    r"\bsupreme court\b", r"\bscotus\b", r"\bjustice department\b", r"\battorney general\b",
    r"\bimpeachment\b", r"\blobby(ist|ing)\b", r"\bpublic policy\b",
    r"\beconom(y|ic|ics)\b", r"\bgdp\b", r"\binflation\b", r"\bunemployment\b",
    r"\bjobs? report\b", r"\bfederal reserve\b", r"\binterest rate(s)?\b",
    r"\bstimulus\b", r"\btrade\b", r"\btariff(s)?\b", r"\bmanufactur(ing|ers?)\b",
    r"\bconsumer confidence\b", r"\bretail sales\b", r"\blabor market\b", r"\bdeficit\b", r"\bnational debt\b",
]
KEYWORD_REGEX = re.compile("|".join(KEYWORD_PATTERNS), re.IGNORECASE)

def _is_domestic(row):
    sec = str(row.get("section") or "").strip().lower()
    if sec in SECTION_ALLOW: return True
    sub = str(row.get("subsection") or "").strip().lower()
    if any(x in sub for x in ["politic","u.s.","business","economy"]): return True
    kws = row.get("keywords", [])
    if isinstance(kws, list):
        if any(KEYWORD_REGEX.search(str(k)) for k in kws): return True
    else:
        if KEYWORD_REGEX.search(str(kws)): return True
    for col in ("headline","abstract"):
        if KEYWORD_REGEX.search(str(row.get(col) or "")): return True
    return False

def filter_domestic_politics_econ(merged: pd.DataFrame) -> pd.DataFrame:
    m = merged.copy()
    mask = m.apply(_is_domestic, axis=1)
    return m[mask].reset_index(drop=True)

def build_dataset(path_2017_2018: str, path_2020: str, out_dir: Optional[str]=None, write_csv: bool=False):
    artsA, commsA = load_2017_2018(path_2017_2018)
    artsB, commsB = load_2020(path_2020)
    mergedA = join_articles_comments(artsA, commsA)
    mergedB = join_articles_comments(artsB, commsB)
    combined = pd.concat([filter_domestic_politics_econ(mergedA), filter_domestic_politics_econ(mergedB)], ignore_index=True)
    if write_csv and out_dir:
        os.makedirs(out_dir, exist_ok=True)
        mergedA.to_csv(os.path.join(out_dir,"joined_2017_2018_all.csv"), index=False)
        mergedB.to_csv(os.path.join(out_dir,"joined_2020_all.csv"), index=False)
        combined.to_csv(os.path.join(out_dir,"domestic_politics_econ_filtered.csv"), index=False)
    return mergedA, mergedB, combined
# --- end inline module ---


In [2]:
# --- Chunked NYT join+filter for Colab (memory-safe) ---
import os, re, glob, math
from typing import List, Optional
import pandas as pd

# --------- helpers ----------
def _clean_url(u):
    if pd.isna(u): return u
    return str(u).strip().split('?')[0].rstrip('/').lower()

def _parse_keywords(val):
    if pd.isna(val): return []
    if isinstance(val, list): return [str(x).strip() for x in val if str(x).strip()]
    return [p.strip() for p in re.split(r'[;,]\s*', str(val)) if p.strip()]

def _to_lower_str(val):
    # Robustly convert to a safe lowercased string (handles pd.NA/NaN/None)
    if val is None: return ""
    if val is pd.NA: return ""
    try:
        if isinstance(val, float) and math.isnan(val):
            return ""
    except Exception:
        pass
    if hasattr(pd, "isna") and pd.isna(val):
        return ""
    return str(val).strip().lower()

ARTICLE_COLS = [
    "article_id","url","pub_date","section","subsection","headline",
    "abstract","news_desk","type_of_material","keywords"
]
COMMENT_COLS = [
    "comment_id","parent_id","user","comment","recommendations","reply_count",
    "editors_selection","date","article_url","article_id"
]

def normalize_articles(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    cand = {
        "article_id":["articleID","asset_id","assetId","id","article_id"],
        "url":["articleURL","url","web_url","link"],
        "pub_date":["pub_date","pubDate","date","published_date","PublicationDate"],
        "section":["section","section_name","newsSection","news_desk"],
        "subsection":["subsection","subsection_name","sub_section"],
        "headline":["headline","title","main_headline","headline.main","Title"],
        "abstract":["abstract","snippet","abstract_text","lead_paragraph","Summary"],
        "news_desk":["news_desk","desk"],
        "type_of_material":["type_of_material","type","material_type_facet"],
        "keywords":["keywords","descriptors","subject","keywords_list"],
    }
    for canon, opts in cand.items():
        for o in opts:
            if o in df.columns:
                df[canon] = df[o]; break
    if "url" in df: df["url"] = df["url"].map(_clean_url)
    if "keywords" in df: df["keywords"] = df["keywords"].map(_parse_keywords)
    else: df["keywords"] = [[] for _ in range(len(df))]
    for c in ARTICLE_COLS:
        if c not in df: df[c] = pd.NA
    return df[ARTICLE_COLS]

def normalize_comments(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    cand = {
        "comment_id":["commentID","comment_id","id"],
        "parent_id":["parentID","parent_id","inReplyTo"],
        "user":["userDisplayName","user","display_name","author"],
        "comment":["commentBody","comment","text","body"],
        "recommendations":["recommendations","recommendCount","recommendedCount"],
        "reply_count":["replyCount","replies","numReplies"],
        "editors_selection":["editorsSelection","editors_selection"],
        "date":["createDate","date","timestamp"],
        "article_url":["articleURL","url","articleUrl","web_url"],
        "article_id":["articleID","asset_id","assetId","article_id"],
    }
    for canon, opts in cand.items():
        for o in opts:
            if o in df.columns:
                df[canon] = df[o]; break
    if "article_url" in df: df["article_url"] = df["article_url"].map(_clean_url)
    for c in COMMENT_COLS:
        if c not in df: df[c] = pd.NA
    return df[COMMENT_COLS]

SECTION_ALLOW = {"u.s.","us","us news","politics","business","economy"}
KEYWORD_PATTERNS = [
    # Politics
    r"\bpolitic(s|al)?\b", r"\bcongress\b", r"\bsenate\b", r"\bhouse of representatives\b",
    r"\bwhite house\b", r"\bpresident\b", r"\bgovern(or|ment)\b", r"\bgubernatorial\b",
    r"\belection(s)?\b", r"\bprimary\b", r"\bmidterm(s)?\b", r"\bballot\b", r"\bcampaign\b",
    r"\bsupreme court\b", r"\bscotus\b", r"\bjustice department\b", r"\battorney general\b",
    r"\bimpeachment\b", r"\blobby(ist|ing)\b", r"\bpublic policy\b",
    # Economy
    r"\beconom(y|ic|ics)\b", r"\bgdp\b", r"\binflation\b", r"\bunemployment\b",
    r"\bjobs? report\b", r"\bfederal reserve\b", r"\binterest rate(s)?\b",
    r"\bstimulus\b", r"\btrade\b", r"\btariff(s)?\b", r"\bmanufactur(ing|ers?)\b",
    r"\bconsumer confidence\b", r"\bretail sales\b", r"\blabor market\b", r"\bdeficit\b", r"\bnational debt\b",
]
KEYWORD_REGEX = re.compile("|".join(KEYWORD_PATTERNS), re.IGNORECASE)

def _is_domestic(row):
    # NA-safe checks
    sec = _to_lower_str(row.get("section"))
    if sec in SECTION_ALLOW:
        return True
    sub = _to_lower_str(row.get("subsection"))
    if any(x in sub for x in ["politic","u.s.","business","economy"]):
        return True

    kws = row.get("keywords", [])
    if isinstance(kws, list):
        for k in kws:
            if KEYWORD_REGEX.search(_to_lower_str(k)):
                return True
    else:
        if KEYWORD_REGEX.search(_to_lower_str(kws)):
            return True

    for col in ("headline","abstract"):
        if KEYWORD_REGEX.search(_to_lower_str(row.get(col))):
            return True
    return False

# --------- dataset loaders (articles fully; comments chunked) ----------
def load_all_articles(dirpath: str) -> pd.DataFrame:
    # articles are relatively small; loading all is fine
    paths = sorted(set(
        glob.glob(os.path.join(dirpath, "Articles*.csv")) +
        glob.glob(os.path.join(dirpath, "*articles*.csv")) +
        glob.glob(os.path.join(dirpath, "articles*.csv"))
    ))
    if not paths:
        raise FileNotFoundError(f"No article CSVs in {dirpath}")
    arts = pd.concat([pd.read_csv(p, low_memory=False) for p in paths], ignore_index=True)
    arts = normalize_articles(arts)
    # Ensure URL is unique-ish; keep first occurrence if duplicates
    arts = arts.sort_values("pub_date").drop_duplicates(subset=["article_id","url"], keep="first")
    return arts

def comment_paths(dirpath: str) -> List[str]:
    paths = sorted(set(
        glob.glob(os.path.join(dirpath, "Comments*.csv")) +
        glob.glob(os.path.join(dirpath, "*comments*.csv")) +
        glob.glob(os.path.join(dirpath, "comments*.csv"))
    ))
    if not paths:
        raise FileNotFoundError(f"No comment CSVs in {dirpath}")
    return paths

def process_comments_chunked(articles: pd.DataFrame, cpaths: List[str], out_joined_csv: Optional[str],
                             out_filtered_csv: str, chunksize: int = 200_000):
    os.makedirs(os.path.dirname(out_filtered_csv), exist_ok=True)
    # Prepare output: remove old files if re-running
    for f in [out_joined_csv, out_filtered_csv]:
        if f and os.path.exists(f):
            os.remove(f)

    # Build fast lookup by URL for fill step (article columns we need)
    art_keep = ["url","article_id","headline","abstract","pub_date","section","subsection","news_desk","type_of_material","keywords"]
    arts_small = articles[art_keep].copy()

    for p in cpaths:
        print(f"Processing comments file: {os.path.basename(p)}")
        for i, chunk in enumerate(pd.read_csv(p, low_memory=False, chunksize=chunksize)):
            c = normalize_comments(chunk)

            # 1) Merge on article_id when present
            m1 = c.merge(articles, how="left", on="article_id", suffixes=("_c","_a"))

            # 2) For rows still missing article info, fill by URL (index-aligned; NA-safe)
            missing = m1["headline"].isna()
            if missing.any():
                missing_idx = m1.index[missing]
                need = m1.loc[missing_idx, ["article_url"]].copy()
                m2 = need.merge(
                    arts_small, how="left", left_on="article_url", right_on="url"
                ).drop(columns=["url"])
                cols_to_fill = ["headline","abstract","pub_date","section","subsection",
                                "news_desk","type_of_material","keywords","article_id"]
                for col in cols_to_fill:
                    target = m1.loc[missing_idx, col]
                    filler = m2[col]
                    to_replace = target.isna()
                    if to_replace.any():
                        m1.loc[missing_idx[to_replace], col] = filler[to_replace].values

            # write joined (optional – big)
            if out_joined_csv:
                m1.to_csv(out_joined_csv, mode="a", header=not os.path.exists(out_joined_csv), index=False)

            # filter and append
            filt = m1[m1.apply(_is_domestic, axis=1)]
            if len(filt):
                filt.to_csv(out_filtered_csv, mode="a", header=not os.path.exists(out_filtered_csv), index=False)

            print(f"  chunk {i+1}: joined={len(m1):,}, kept={len(filt):,}")

# --------- master runner ----------
def build_dataset_chunked(path_17_18: str, path_2020: str, out_dir="nyt_outputs", chunksize=200_000,
                          write_joined=False):
    os.makedirs(out_dir, exist_ok=True)

    # 2017/2018
    artsA = load_all_articles(path_17_18)
    cpathsA = comment_paths(path_17_18)
    joinedA = os.path.join(out_dir, "joined_2017_2018_all.csv") if write_joined else None
    filtA   = os.path.join(out_dir, "domestic_politics_econ_2017_2018.csv")
    process_comments_chunked(artsA, cpathsA, joinedA, filtA, chunksize=chunksize)

    # 2020
    artsB = load_all_articles(path_2020)
    cpathsB = comment_paths(path_2020)
    joinedB = os.path.join(out_dir, "joined_2020_all.csv") if write_joined else None
    filtB   = os.path.join(out_dir, "domestic_politics_econ_2020.csv")
    process_comments_chunked(artsB, cpathsB, joinedB, filtB, chunksize=chunksize)

    # Combine filtered pieces into a single file
    combo_out = os.path.join(out_dir, "domestic_politics_econ_filtered.csv")
    if os.path.exists(combo_out): os.remove(combo_out)
    for f in [filtA, filtB]:
        if os.path.exists(f):
            pd.read_csv(f, low_memory=False).to_csv(
                combo_out, mode="a", header=not os.path.exists(combo_out), index=False
            )
    print("Done. Outputs in:", out_dir)


In [3]:
import kagglehub

path_17_18 = kagglehub.dataset_download("aashita/nyt-comments")
path_2020  = kagglehub.dataset_download("benjaminawd/new-york-times-articles-comments-2020")




Using Colab cache for faster access to the 'nyt-comments' dataset.
Using Colab cache for faster access to the 'new-york-times-articles-comments-2020' dataset.


In [4]:
import pandas as pd
import math

def _to_lower_str(val):
    # robustly convert any value (including pd.NA/NaN/None) to a safe lowercased string
    if val is None or (isinstance(val, float) and math.isnan(val)):
        return ""
    if val is pd.NA or (hasattr(pd, "isna") and pd.isna(val)):
        return ""
    return str(val).strip().lower()

def _is_domestic(row):
    # section/subsection checks
    sec = _to_lower_str(row.get("section"))
    if sec in {"u.s.", "us", "us news", "politics", "business", "economy"}:
        return True

    sub = _to_lower_str(row.get("subsection"))
    if any(x in sub for x in ["politic", "u.s.", "business", "economy"]):
        return True

    # keywords/descriptors (could be list, string, or NA)
    kws = row.get("keywords", [])
    if isinstance(kws, list):
        for k in kws:
            txt = _to_lower_str(k)
            if KEYWORD_REGEX.search(txt):
                return True
    else:
        txt = _to_lower_str(kws)
        if KEYWORD_REGEX.search(txt):
            return True

    # headline/abstract fallback
    for col in ("headline", "abstract"):
        txt = _to_lower_str(row.get(col))
        if KEYWORD_REGEX.search(txt):
            return True

    return False


In [ ]:
build_dataset_chunked(
    path_17_18,
    path_2020,
    out_dir="/content/nyt_outputs",
    chunksize=50_000,   # lower to 100_000 if memory is tight
    write_joined=False
)


Processing comments file: CommentsApril2017.csv
  chunk 1: joined=50,000, kept=26,942
  chunk 2: joined=50,000, kept=21,229
  chunk 3: joined=50,000, kept=32,515
  chunk 4: joined=50,000, kept=31,170
  chunk 5: joined=43,832, kept=22,714
Processing comments file: CommentsApril2018.csv
  chunk 1: joined=50,000, kept=30,713
  chunk 2: joined=50,000, kept=38,189


In [1]:
# --- ULTRA-LOW-MEM NYT join+filter (dict maps; no big merges) ---
import os, re, glob, math, gc
from typing import List, Optional, Dict, Tuple
import pandas as pd

# --------- helpers ----------
def _clean_url(u):
    if pd.isna(u): return u
    return str(u).strip().split('?')[0].rstrip('/').lower()

def _to_lower_str(val):
    if val is None or val is pd.NA: return ""
    try:
        if isinstance(val, float) and math.isnan(val): return ""
    except Exception:
        pass
    if hasattr(pd, "isna") and pd.isna(val): return ""
    return str(val).strip().lower()

SECTION_ALLOW = {"u.s.","us","us news","politics","business","economy"}
KEYWORD_PATTERNS = [
    # Politics
    r"\bpolitic(s|al)?\b", r"\bcongress\b", r"\bsenate\b", r"\bhouse of representatives\b",
    r"\bwhite house\b", r"\bpresident\b", r"\bgovern(or|ment)\b", r"\bgubernatorial\b",
    r"\belection(s)?\b", r"\bprimary\b", r"\bmidterm(s)?\b", r"\bballot\b", r"\bcampaign\b",
    r"\bsupreme court\b", r"\bscotus\b", r"\bjustice department\b", r"\battorney general\b",
    r"\bimpeachment\b", r"\blobby(ist|ing)\b", r"\bpublic policy\b",
    # Economy
    r"\beconom(y|ic|ics)\b", r"\bgdp\b", r"\binflation\b", r"\bunemployment\b",
    r"\bjobs? report\b", r"\bfederal reserve\b", r"\binterest rate(s)?\b",
    r"\bstimulus\b", r"\btrade\b", r"\btariff(s)?\b", r"\bmanufactur(ing|ers?)\b",
    r"\bconsumer confidence\b", r"\bretail sales\b", r"\blabor market\b", r"\bdeficit\b", r"\bnational debt\b",
]
KEYWORD_REGEX = re.compile("|".join(KEYWORD_PATTERNS), re.IGNORECASE)

# Columns we want to preserve in output
OUT_COLS = [
    # comment-side
    "comment_id","parent_id","user","comment","recommendations","reply_count",
    "editors_selection","date",
    # article-side
    "article_id","article_url","url","pub_date","section","subsection","headline",
    "abstract","news_desk","type_of_material","keywords",
]

# Flexible name maps for normalization (minimal set)
ART_MAP = {
    "article_id": ["articleID","asset_id","assetId","id","article_id"],
    "url": ["articleURL","url","web_url","link"],
    "pub_date": ["pub_date","pubDate","date","published_date","PublicationDate"],
    "section": ["section","section_name","newsSection","news_desk"],
    "subsection": ["subsection","subsection_name","sub_section"],
    "headline": ["headline","title","main_headline","headline.main","Title"],
    "abstract": ["abstract","snippet","abstract_text","lead_paragraph","Summary"],
    "news_desk": ["news_desk","desk"],
    "type_of_material": ["type_of_material","type","material_type_facet"],
    "keywords": ["keywords","descriptors","subject","keywords_list"],
}

COM_MAP = {
    "comment_id": ["commentID","comment_id","id"],
    "parent_id": ["parentID","parent_id","inReplyTo"],
    "user": ["userDisplayName","user","display_name","author"],
    "comment": ["commentBody","comment","text","body"],
    "recommendations": ["recommendations","recommendCount","recommendedCount"],
    "reply_count": ["replyCount","replies","numReplies"],
    "editors_selection": ["editorsSelection","editors_selection"],
    "date": ["createDate","date","timestamp"],
    "article_url": ["articleURL","url","articleUrl","web_url"],
    "article_id": ["articleID","asset_id","assetId","article_id"],
}

def _first_present(df, candidates, default=pd.NA):
    for c in candidates:
        if c in df.columns:
            return df[c]
    return pd.Series([default]*len(df))

def _normalize_articles_small(df: pd.DataFrame) -> pd.DataFrame:
    out = pd.DataFrame({
        "article_id": _first_present(df, ART_MAP["article_id"]),
        "url": _first_present(df, ART_MAP["url"]).map(_clean_url),
        "pub_date": _first_present(df, ART_MAP["pub_date"]),
        "section": _first_present(df, ART_MAP["section"]),
        "subsection": _first_present(df, ART_MAP["subsection"]),
        "headline": _first_present(df, ART_MAP["headline"]),
        "abstract": _first_present(df, ART_MAP["abstract"]),
        "news_desk": _first_present(df, ART_MAP["news_desk"]),
        "type_of_material": _first_present(df, ART_MAP["type_of_material"]),
        "keywords": _first_present(df, ART_MAP["keywords"]),
    })
    return out

def _normalize_comments_small(df: pd.DataFrame) -> pd.DataFrame:
    out = pd.DataFrame({
        "comment_id": _first_present(df, COM_MAP["comment_id"]),
        "parent_id": _first_present(df, COM_MAP["parent_id"]),
        "user": _first_present(df, COM_MAP["user"]),
        "comment": _first_present(df, COM_MAP["comment"]),
        "recommendations": _first_present(df, COM_MAP["recommendations"]),
        "reply_count": _first_present(df, COM_MAP["reply_count"]),
        "editors_selection": _first_present(df, COM_MAP["editors_selection"]),
        "date": _first_present(df, COM_MAP["date"]),
        "article_url": _first_present(df, COM_MAP["article_url"]).map(_clean_url),
        "article_id": _first_present(df, COM_MAP["article_id"]),
    })
    return out

def _is_domestic_row(section, subsection, headline, abstract, keywords):
    sec = _to_lower_str(section)
    if sec in SECTION_ALLOW:
        return True
    sub = _to_lower_str(subsection)
    if any(x in sub for x in ["politic","u.s.","business","economy"]):
        return True
    # keywords may be list-like or string; treat as string
    if isinstance(keywords, list):
        kwtxt = " ".join(_to_lower_str(k) for k in keywords if k is not None)
    else:
        kwtxt = _to_lower_str(keywords)
    if KEYWORD_REGEX.search(kwtxt or ""):
        return True
    if KEYWORD_REGEX.search(_to_lower_str(headline) or ""):
        return True
    if KEYWORD_REGEX.search(_to_lower_str(abstract) or ""):
        return True
    return False

# --------- dataset loaders ----------
def _find(paths_patterns: List[str]) -> List[str]:
    s = set()
    for pat in paths_patterns:
        s.update(glob.glob(pat))
    return sorted(s)

def load_articles_build_maps(dirpath: str):
    a_paths = _find([
        os.path.join(dirpath, "Articles*.csv"),
        os.path.join(dirpath, "*articles*.csv"),
        os.path.join(dirpath, "articles*.csv"),
    ])
    if not a_paths:
        raise FileNotFoundError(f"No article CSVs in {dirpath}")

    arts = pd.concat([pd.read_csv(p, low_memory=False) for p in a_paths], ignore_index=True)
    arts = _normalize_articles_small(arts)

    # Drop dups, keep earliest by pub_date
    arts = arts.sort_values("pub_date").drop_duplicates(subset=["article_id","url"], keep="first")

    # Build compact tuples for map
    cols = ["headline","abstract","pub_date","section","subsection","news_desk","type_of_material","keywords","url"]
    tup = arts[cols].itertuples(index=False, name=None)

    by_id: Dict[str, Tuple] = {}
    if "article_id" in arts.columns:
        for aid, t in zip(arts["article_id"], tup):
            if pd.isna(aid): continue
            by_id[str(aid)] = t
    # rebuild tup iterator for url map
    tup2 = arts[cols].itertuples(index=False, name=None)
    by_url: Dict[str, Tuple] = {}
    for u, t in zip(arts["url"], tup2):
        if pd.isna(u): continue
        by_url[str(u)] = t

    return by_id, by_url

def comment_files(dirpath: str) -> List[str]:
    c_paths = _find([
        os.path.join(dirpath, "Comments*.csv"),
        os.path.join(dirpath, "*comments*.csv"),
        os.path.join(dirpath, "comments*.csv"),
    ])
    if not c_paths:
        raise FileNotFoundError(f"No comment CSVs in {dirpath}")
    return c_paths

# --------- streaming join/filter ----------
def process_comment_stream(by_id, by_url, cpaths: List[str], out_filtered_csv: str,
                           chunksize: int = 25_000):
    os.makedirs(os.path.dirname(out_filtered_csv), exist_ok=True)
    if os.path.exists(out_filtered_csv):
        os.remove(out_filtered_csv)

    # Only read the minimal set of columns we care about
    allowed = set(sum(COM_MAP.values(), []))  # flatten list of lists
    usecols = lambda c: c in allowed  # callable avoids errors if some don't exist

    for p in cpaths:
        print(f"Processing: {os.path.basename(p)}")
        for i, raw in enumerate(pd.read_csv(p, low_memory=False, chunksize=chunksize, usecols=usecols)):
            c = _normalize_comments_small(raw)

            # Prepare article fields from ID map
            aid = c["article_id"].astype(str)
            got = aid.map(by_id)  # tuple or NaN

            # Create empty columns
            for col in ["headline","abstract","pub_date","section","subsection","news_desk","type_of_material","keywords","url"]:
                c[col] = pd.NA

            # Fill from ID hits
            mask_id = got.notna()
            if mask_id.any():
                vals = got[mask_id].tolist()
                cols = ["headline","abstract","pub_date","section","subsection","news_desk","type_of_material","keywords","url"]
                filled = pd.DataFrame(vals, index=c.index[mask_id], columns=cols)
                for col in cols:
                    c.loc[mask_id, col] = filled[col].values

            # For remaining rows, try URL lookup
            mask_need = c["headline"].isna()
            if mask_need.any():
                url_clean = c.loc[mask_need, "article_url"].map(lambda x: _clean_url(x))
                got2 = url_clean.map(by_url)
                mask_url = got2.notna()
                if mask_url.any():
                    vals2 = got2[mask_url].tolist()
                    filled2 = pd.DataFrame(vals2, index=c.index[mask_need][mask_url], columns=cols)
                    for col in cols:
                        c.loc[c.index[mask_need][mask_url], col] = filled2[col].values

            # Filter rows for domestic politics/economy
            keep_mask = c.apply(
                lambda r: _is_domestic_row(r["section"], r["subsection"], r["headline"], r["abstract"], r["keywords"]),
                axis=1
            )
            out = c.loc[keep_mask, OUT_COLS].copy()

            if len(out):
                out.to_csv(out_filtered_csv, mode="a", header=not os.path.exists(out_filtered_csv), index=False)

            print(f"  chunk {i+1}: total={len(c):,}, kept={len(out):,}")
            # free memory
            del raw, c, out, got
            if 'got2' in locals(): del got2
            gc.collect()

def build_dataset_ultra(path_2017_2018: str, path_2020: str,
                        out_dir="/content/nyt_outputs", chunksize=25_000):
    os.makedirs(out_dir, exist_ok=True)

    # 2017/2018
    by_id_A, by_url_A = load_articles_build_maps(path_2017_2018)
    filtA = os.path.join(out_dir, "domestic_politics_econ_2017_2018.csv")
    process_comment_stream(by_id_A, by_url_A, comment_files(path_2017_2018), filtA, chunksize=chunksize)

    # 2020
    by_id_B, by_url_B = load_articles_build_maps(path_2020)
    filtB = os.path.join(out_dir, "domestic_politics_econ_2020.csv")
    process_comment_stream(by_id_B, by_url_B, comment_files(path_2020), filtB, chunksize=chunksize)

    # combine
    combo = os.path.join(out_dir, "domestic_politics_econ_filtered.csv")
    if os.path.exists(combo): os.remove(combo)
    for f in [filtA, filtB]:
        if os.path.exists(f):
            pd.read_csv(f, low_memory=False).to_csv(combo, mode="a", header=not os.path.exists(combo), index=False)

    print("Done. Outputs in:", out_dir)


In [2]:
import kagglehub

path_17_18 = kagglehub.dataset_download("aashita/nyt-comments")
path_2020  = kagglehub.dataset_download("benjaminawd/new-york-times-articles-comments-2020")

build_dataset_ultra(
    path_2017_2018=path_17_18,
    path_2020=path_2020,
    out_dir="/content/nyt_outputs",
    chunksize=25_000  # try 10_000 if T4 is still tight
)


Using Colab cache for faster access to the 'nyt-comments' dataset.
Using Colab cache for faster access to the 'new-york-times-articles-comments-2020' dataset.
Processing: CommentsApril2017.csv
  chunk 1: total=25,000, kept=14,057
  chunk 2: total=50,000, kept=12,885
  chunk 3: total=50,000, kept=12,288
  chunk 4: total=50,000, kept=8,941
  chunk 5: total=50,000, kept=15,331
  chunk 6: total=50,000, kept=17,184
  chunk 7: total=50,000, kept=18,535
  chunk 8: total=50,000, kept=12,635
  chunk 9: total=50,000, kept=13,714
  chunk 10: total=37,664, kept=9,000
Processing: CommentsApril2018.csv
  chunk 1: total=25,000, kept=16,110
  chunk 2: total=50,000, kept=14,603
  chunk 3: total=50,000, kept=19,113
  chunk 4: total=50,000, kept=19,076
  chunk 5: total=50,000, kept=12,122
  chunk 6: total=50,000, kept=15,023
  chunk 7: total=50,000, kept=11,398
  chunk 8: total=50,000, kept=13,867
  chunk 9: total=50,000, kept=11,870
  chunk 10: total=50,000, kept=11,313
  chunk 11: total=29,848, kept=8,

KeyboardInterrupt: 

In [1]:
# --- ULTRA-LOW-MEM NYT join+filter (robust ID/URL maps + URL-aware filter) ---
import os, re, glob, math, gc
from typing import List, Optional, Dict, Tuple
import pandas as pd

# --------- helpers ----------
def _clean_url(u):
    if pd.isna(u): return u
    return str(u).strip().split('?')[0].rstrip('/').lower()

def _to_lower_str(val):
    if val is None or val is pd.NA: return ""
    try:
        if isinstance(val, float) and math.isnan(val): return ""
    except Exception:
        pass
    if hasattr(pd, "isna") and pd.isna(val): return ""
    return str(val).strip().lower()

SECTION_ALLOW = {"u.s.","us","us news","politics","business","economy"}
KEYWORD_PATTERNS = [
    # Politics
    r"\bpolitic(s|al)?\b", r"\bcongress\b", r"\bsenate\b", r"\bhouse of representatives\b",
    r"\bwhite house\b", r"\bpresident\b", r"\bgovern(or|ment)\b", r"\bgubernatorial\b",
    r"\belection(s)?\b", r"\bprimary\b", r"\bmidterm(s)?\b", r"\bballot\b", r"\bcampaign\b",
    r"\bsupreme court\b", r"\bscotus\b", r"\bjustice department\b", r"\battorney general\b",
    r"\bimpeachment\b", r"\blobby(ist|ing)\b", r"\bpublic policy\b",
    # Economy
    r"\beconom(y|ic|ics)\b", r"\bgdp\b", r"\binflation\b", r"\bunemployment\b",
    r"\bjobs? report\b", r"\bfederal reserve\b", r"\binterest rate(s)?\b",
    r"\bstimulus\b", r"\btrade\b", r"\btariff(s)?\b", r"\bmanufactur(ing|ers?)\b",
    r"\bconsumer confidence\b", r"\bretail sales\b", r"\blabor market\b", r"\bdeficit\b", r"\bnational debt\b",
]
KEYWORD_REGEX = re.compile("|".join(KEYWORD_PATTERNS), re.IGNORECASE)

# Columns we want to preserve in output
OUT_COLS = [
    # comment-side
    "comment_id","parent_id","user","comment","recommendations","reply_count",
    "editors_selection","date",
    # article-side
    "article_id","article_url","url","pub_date","section","subsection","headline",
    "abstract","news_desk","type_of_material","keywords",
]

# Flexible name maps for normalization (minimal set)
ART_MAP = {
    "article_id": ["articleID","asset_id","assetId","id","article_id"],
    "url": ["articleURL","url","web_url","link"],
    "pub_date": ["pub_date","pubDate","date","published_date","PublicationDate"],
    "section": ["section","section_name","newsSection","news_desk"],
    "subsection": ["subsection","subsection_name","sub_section"],
    "headline": ["headline","title","main_headline","headline.main","Title"],
    "abstract": ["abstract","snippet","abstract_text","lead_paragraph","Summary"],
    "news_desk": ["news_desk","desk"],
    "type_of_material": ["type_of_material","type","material_type_facet"],
    "keywords": ["keywords","descriptors","subject","keywords_list"],
}

COM_MAP = {
    "comment_id": ["commentID","comment_id","id"],
    "parent_id": ["parentID","parent_id","inReplyTo"],
    "user": ["userDisplayName","user","display_name","author"],
    "comment": ["commentBody","comment","text","body"],
    "recommendations": ["recommendations","recommendCount","recommendedCount"],
    "reply_count": ["replyCount","replies","numReplies"],
    "editors_selection": ["editorsSelection","editors_selection"],
    "date": ["createDate","date","timestamp"],
    "article_url": ["articleURL","url","articleUrl","web_url"],
    "article_id": ["articleID","asset_id","assetId","article_id"],
}

def _first_present(df, candidates, default=pd.NA):
    for c in candidates:
        if c in df.columns:
            return df[c]
    return pd.Series([default]*len(df))

def _normalize_articles_small(df: pd.DataFrame) -> pd.DataFrame:
    out = pd.DataFrame({
        "article_id": _first_present(df, ART_MAP["article_id"]),
        "url": _first_present(df, ART_MAP["url"]).map(_clean_url),
        "pub_date": _first_present(df, ART_MAP["pub_date"]),
        "section": _first_present(df, ART_MAP["section"]),
        "subsection": _first_present(df, ART_MAP["subsection"]),
        "headline": _first_present(df, ART_MAP["headline"]),
        "abstract": _first_present(df, ART_MAP["abstract"]),
        "news_desk": _first_present(df, ART_MAP["news_desk"]),
        "type_of_material": _first_present(df, ART_MAP["type_of_material"]),
        "keywords": _first_present(df, ART_MAP["keywords"]),
    })
    return out

def _normalize_comments_small(df: pd.DataFrame) -> pd.DataFrame:
    out = pd.DataFrame({
        "comment_id": _first_present(df, COM_MAP["comment_id"]),
        "parent_id": _first_present(df, COM_MAP["parent_id"]),
        "user": _first_present(df, COM_MAP["user"]),
        "comment": _first_present(df, COM_MAP["comment"]),
        "recommendations": _first_present(df, COM_MAP["recommendations"]),
        "reply_count": _first_present(df, COM_MAP["reply_count"]),
        "editors_selection": _first_present(df, COM_MAP["editors_selection"]),
        "date": _first_present(df, COM_MAP["date"]),
        "article_url": _first_present(df, COM_MAP["article_url"]).map(_clean_url),
        "article_id": _first_present(df, COM_MAP["article_id"]),
    })
    return out

def _is_domestic_row(section, subsection, headline, abstract, keywords, url_hint=None, url_hint2=None):
    def url_says_domestic(u):
        u = _to_lower_str(u)
        if not u:
            return False
        # section paths
        if any(seg in u for seg in ["/politics/", "/us/", "/business/", "/economy/"]):
            return True
        # slug keywords
        if any(tok in u for tok in [
            "supreme-court", "congress", "senate", "house-of-representatives", "white-house",
            "president", "governor", "election", "primary", "midterm", "ballot", "campaign",
            "federal-reserve", "inflation", "unemployment", "jobs-report", "interest-rate",
            "stimulus", "trade", "tariff", "manufactur", "consumer-confidence", "retail-sales",
            "labor-market", "deficit", "national-debt"
        ]):
            return True
        return False

    # 1) Section/subsection checks
    sec = _to_lower_str(section)
    if sec in SECTION_ALLOW:
        return True
    sub = _to_lower_str(subsection)
    if any(x in sub for x in ["politic","u.s.","business","economy"]):
        return True

    # 2) Keywords/headline/abstract regex
    if isinstance(keywords, list):
        kwtxt = " ".join(_to_lower_str(k) for k in keywords if k is not None)
    else:
        kwtxt = _to_lower_str(keywords)
    if KEYWORD_REGEX.search(kwtxt or ""):
        return True
    if KEYWORD_REGEX.search(_to_lower_str(headline) or ""):
        return True
    if KEYWORD_REGEX.search(_to_lower_str(abstract) or ""):
        return True

    # 3) URL-based fallback
    if url_says_domestic(url_hint) or url_says_domestic(url_hint2):
        return True

    return False

# --------- dataset loaders ----------
def _find(paths_patterns: List[str]) -> List[str]:
    s = set()
    for pat in paths_patterns:
        s.update(glob.glob(pat))
    return sorted(s)

def load_articles_build_maps(dirpath: str):
    a_paths = _find([
        os.path.join(dirpath, "Articles*.csv"),
        os.path.join(dirpath, "*articles*.csv"),
        os.path.join(dirpath, "articles*.csv"),
    ])
    if not a_paths:
        raise FileNotFoundError(f"No article CSVs in {dirpath}")

    arts = pd.concat([pd.read_csv(p, low_memory=False) for p in a_paths], ignore_index=True)
    arts = _normalize_articles_small(arts)

    # Drop dups, keep earliest by pub_date
    arts = arts.sort_values("pub_date").drop_duplicates(subset=["article_id","url"], keep="first")

    cols = ["headline","abstract","pub_date","section","subsection","news_desk","type_of_material","keywords","url"]

    # Build ID map with BOTH raw and str keys
    by_id: Dict[object, Tuple] = {}
    for aid, t in zip(arts["article_id"], arts[cols].itertuples(index=False, name=None)):
        if pd.isna(aid): continue
        by_id[aid] = t
    for aid, t in zip(arts["article_id"], arts[cols].itertuples(index=False, name=None)):
        if pd.isna(aid): continue
        by_id[str(aid)] = t  # string key variant

    # URL map (cleaned)
    by_url: Dict[str, Tuple] = {}
    for u, t in zip(arts["url"], arts[cols].itertuples(index=False, name=None)):
        if pd.isna(u): continue
        by_url[str(u)] = t

    return by_id, by_url

def comment_files(dirpath: str) -> List[str]:
    c_paths = _find([
        os.path.join(dirpath, "Comments*.csv"),
        os.path.join(dirpath, "*comments*.csv"),
        os.path.join(dirpath, "comments*.csv"),
    ])
    if not c_paths:
        raise FileNotFoundError(f"No comment CSVs in {dirpath}")
    return c_paths

# --------- streaming join/filter ----------
def process_comment_stream(by_id, by_url, cpaths: List[str], out_filtered_csv: str,
                           chunksize: int = 25_000):
    os.makedirs(os.path.dirname(out_filtered_csv), exist_ok=True)
    if os.path.exists(out_filtered_csv):
        os.remove(out_filtered_csv)

    # Only read the minimal set of columns we care about
    allowed = set(sum(COM_MAP.values(), []))  # flatten list of lists
    usecols = lambda c: c in allowed  # callable avoids errors if some don't exist

    for p in cpaths:
        print(f"Processing: {os.path.basename(p)}")
        for i, raw in enumerate(pd.read_csv(p, low_memory=False, chunksize=chunksize, usecols=usecols)):
            c = _normalize_comments_small(raw)

            # Prepare empty article columns
            for col in ["headline","abstract","pub_date","section","subsection","news_desk","type_of_material","keywords","url"]:
                c[col] = pd.NA

            cols = ["headline","abstract","pub_date","section","subsection","news_desk","type_of_material","keywords","url"]

            # --- ID lookups (try raw first, then str as fallback) ---
            aid_raw = c["article_id"]
            got_raw = aid_raw.map(by_id)            # matches when types align
            mask_raw = got_raw.notna()
            id_hits_raw = int(mask_raw.sum())
            if id_hits_raw:
                vals = got_raw[mask_raw].tolist()
                filled = pd.DataFrame(vals, index=c.index[mask_raw], columns=cols)
                for col in cols:
                    c.loc[mask_raw, col] = filled[col].values

            mask_need_id = c["headline"].isna()
            got_str = aid_raw.astype(str).where(mask_need_id, None).map(by_id)
            mask_str = got_str.notna()
            id_hits_str = int(mask_str.sum())
            if id_hits_str:
                vals2 = got_str[mask_str].tolist()
                filled2 = pd.DataFrame(vals2, index=c.index[mask_str], columns=cols)
                for col in cols:
                    c.loc[mask_str, col] = filled2[col].values

            # --- URL fallback ---
            mask_need_url = c["headline"].isna()
            url_clean = c.loc[mask_need_url, "article_url"].map(_clean_url)
            got_url = url_clean.map(by_url)
            mask_url = got_url.notna()
            url_hits = int(mask_url.sum())
            if url_hits:
                vals3 = got_url[mask_url].tolist()
                filled3 = pd.DataFrame(vals3, index=c.index[mask_need_url][mask_url], columns=cols)
                for col in cols:
                    c.loc[c.index[mask_need_url][mask_url], col] = filled3[col].values

            # --- Filtering (pass both possible URL hints) ---
            keep_mask = c.apply(
                lambda r: _is_domestic_row(
                    r["section"], r["subsection"], r["headline"], r["abstract"], r["keywords"],
                    url_hint=r.get("url"), url_hint2=r.get("article_url")
                ),
                axis=1
            )
            out = c.loc[keep_mask, OUT_COLS].copy()

            if len(out):
                out.to_csv(out_filtered_csv, mode="a", header=not os.path.exists(out_filtered_csv), index=False)

            print(f"  chunk {i+1}: total={len(c):,}, id_hits={id_hits_raw + id_hits_str:,}, url_hits={url_hits:,}, kept={len(out):,}")

            # free memory
            del raw, c, out, got_raw, got_str, got_url
            gc.collect()

def build_dataset_ultra(path_2017_2018: str, path_2020: str,
                        out_dir="/content/nyt_outputs", chunksize=25_000):
    os.makedirs(out_dir, exist_ok=True)

    # 2017/2018
    by_id_A, by_url_A = load_articles_build_maps(path_2017_2018)
    filtA = os.path.join(out_dir, "domestic_politics_econ_2017_2018.csv")
    process_comment_stream(by_id_A, by_url_A, comment_files(path_2017_2018), filtA, chunksize=chunksize)

    # 2020
    by_id_B, by_url_B = load_articles_build_maps(path_2020)
    filtB = os.path.join(out_dir, "domestic_politics_econ_2020.csv")
    process_comment_stream(by_id_B, by_url_B, comment_files(path_2020), filtB, chunksize=chunksize)

    # combine
    combo = os.path.join(out_dir, "domestic_politics_econ_filtered.csv")
    if os.path.exists(combo): os.remove(combo)
    for f in [filtA, filtB]:
        if os.path.exists(f):
            pd.read_csv(f, low_memory=False).to_csv(combo, mode="a", header=not os.path.exists(combo), index=False)

    print("Done. Outputs in:", out_dir)


In [2]:
import kagglehub

path_17_18 = kagglehub.dataset_download("aashita/nyt-comments")
path_2020  = kagglehub.dataset_download("benjaminawd/new-york-times-articles-comments-2020")

build_dataset_ultra(
    path_2017_2018=path_17_18,
    path_2020=path_2020,
    out_dir="/content/nyt_outputs",
    chunksize=25_000   # try 10_000 if memory is tight
)


Using Colab cache for faster access to the 'nyt-comments' dataset.
Using Colab cache for faster access to the 'new-york-times-articles-comments-2020' dataset.
Processing: CommentsApril2017.csv
  chunk 1: total=25,000, id_hits=25,000, url_hits=0, kept=14,057
  chunk 2: total=50,000, id_hits=25,000, url_hits=0, kept=12,885
  chunk 3: total=50,000, id_hits=25,000, url_hits=0, kept=12,288
  chunk 4: total=50,000, id_hits=25,000, url_hits=0, kept=8,941
  chunk 5: total=50,000, id_hits=25,000, url_hits=0, kept=15,331
  chunk 6: total=50,000, id_hits=25,000, url_hits=0, kept=17,184
  chunk 7: total=50,000, id_hits=25,000, url_hits=0, kept=18,535
  chunk 8: total=50,000, id_hits=25,000, url_hits=0, kept=12,635
  chunk 9: total=50,000, id_hits=25,000, url_hits=0, kept=13,714
  chunk 10: total=37,664, id_hits=18,832, url_hits=0, kept=9,000
Processing: CommentsApril2018.csv
  chunk 1: total=25,000, id_hits=25,000, url_hits=0, kept=16,110
  chunk 2: total=50,000, id_hits=25,000, url_hits=0, kept=1

KeyboardInterrupt: 

In [3]:
# --- Ultra-low-memory NYT election-only pipeline (Colab friendly) ---
import os, re, glob, math, gc
from typing import List, Optional, Dict, Tuple
import pandas as pd

# ------------------- Helpers -------------------
def _clean_url(u):
    if pd.isna(u): return u
    return str(u).strip().split('?')[0].rstrip('/').lower()

def _to_lower_str(val):
    if val is None or val is pd.NA: return ""
    try:
        if isinstance(val, float) and math.isnan(val): return ""
    except Exception:
        pass
    if hasattr(pd, "isna") and pd.isna(val): return ""
    return str(val).strip().lower()

# Output columns
OUT_COLS = [
    "comment_id","parent_id","user","comment","recommendations","reply_count",
    "editors_selection","date",
    "article_id","article_url","url","pub_date","section","subsection","headline",
    "abstract","news_desk","type_of_material","keywords",
]

# ------------------- Column maps -------------------
# Articles: prefer uniqueID
ART_MAP = {
    "article_id": ["uniqueID","uniqueid","uniqueId","UniqueID","articleID","asset_id","assetId","id","article_id"],
    "url": ["articleURL","url","web_url","link"],
    "pub_date": ["pub_date","pubDate","date","published_date","PublicationDate"],
    "section": ["section","section_name","newsSection","news_desk"],
    "subsection": ["subsection","subsection_name","sub_section"],
    "headline": ["headline","title","main_headline","headline.main","Title"],
    "abstract": ["abstract","snippet","abstract_text","lead_paragraph","Summary"],
    "news_desk": ["news_desk","desk"],
    "type_of_material": ["type_of_material","type","material_type_facet"],
    "keywords": ["keywords","descriptors","subject","keywords_list"],
}

# Comments: prefer articleID
COM_MAP = {
    "comment_id": ["commentID","comment_id","id"],
    "parent_id": ["parentID","parent_id","inReplyTo"],
    "user": ["userDisplayName","user","display_name","author"],
    "comment": ["commentBody","comment","text","body","content"],
    "recommendations": ["recommendations","recommendCount","recommendedCount","recommendationCount"],
    "reply_count": ["replyCount","replies","numReplies"],
    "editors_selection": ["editorsSelection","editors_selection"],
    "date": ["createDate","date","timestamp","createdAt"],
    "article_url": ["articleURL","url","web_url","articleUrl","story_url","storyUrl","article_link","articleLink","link"],
    "article_id": ["articleID","articleid","asset_id","assetId","article_id","story_id","storyId"],
}

# ------------------- Election filtering -------------------
ELECTION_KEYWORD_PATTERNS = [
    r"\belection(s)?\b", r"\bprimary\b", r"\bcaucus(es)?\b", r"\brunoff\b",
    r"\bmidterm(s)?\b", r"\bgeneral election\b",
    r"\bballot(s)?\b", r"\babsentee\b", r"\bmail[- ]in\b", r"\bearly voting\b",
    r"\bvoter(s)?\b", r"\bvoter registration\b", r"\bregistered voter(s)?\b",
    r"\bvoting\b", r"\bpoll(s|ing place)?\b", r"\bturnout\b",
    r"\bcampaign(s|ing)?\b", r"\bcandidate(s)?\b",
    r"\bpac(s)?\b", r"\bsuper pac(s)?\b", r"\bfec\b",
    r"\bdebate(s)?\b", r"\bendorse(ment|ments)?\b",
]
ELECTION_REGEX = re.compile("|".join(ELECTION_KEYWORD_PATTERNS), re.IGNORECASE)

ELECTION_URL_SEGMENTS = [
    "/politics/elections", "/elections/", "/interactive/us/elections",
    "/politics/", "/us/politics/", "/live/20",
    "/news-event/2020-election", "/news-event/2022-midterms", "/news-event/2024-election"
]

ELECTION_SECTIONS = {"politics"}
ELECTION_SUBSTR = ["election", "campaign", "midterm", "primary", "vote"]

def _is_us_election_row(section, subsection, headline, abstract, keywords, url_hint=None, url_hint2=None):
    def url_says_election(u):
        u = _to_lower_str(u)
        if not u:
            return False
        if any(seg in u for seg in ELECTION_URL_SEGMENTS):
            return True
        if any(tok in u for tok in [
            "election", "primary", "caucus", "midterm", "ballot", "campaign",
            "voting", "turnout", "polls"
        ]):
            return True
        return False

    # Section/subsection
    sec = _to_lower_str(section)
    if sec in ELECTION_SECTIONS:
        return True
    sub = _to_lower_str(subsection)
    if any(x in sub for x in ELECTION_SUBSTR):
        return True

    # Keywords/headline/abstract
    if isinstance(keywords, list):
        kwtxt = " ".join(_to_lower_str(k) for k in keywords if k is not None)
    else:
        kwtxt = _to_lower_str(keywords)
    if ELECTION_REGEX.search(kwtxt or ""):
        return True
    if ELECTION_REGEX.search(_to_lower_str(headline) or ""):
        return True
    if ELECTION_REGEX.search(_to_lower_str(abstract) or ""):
        return True

    # URL fallback
    if url_says_election(url_hint) or url_says_election(url_hint2):
        return True

    return False

# ------------------- Normalizers -------------------
def _first_present(df, candidates, default=pd.NA):
    for c in candidates:
        if c in df.columns:
            return df[c]
    return pd.Series([default]*len(df))

def _normalize_articles_small(df: pd.DataFrame) -> pd.DataFrame:
    out = pd.DataFrame({
        "article_id": _first_present(df, ART_MAP["article_id"]),
        "url": _first_present(df, ART_MAP["url"]).map(_clean_url),
        "pub_date": _first_present(df, ART_MAP["pub_date"]),
        "section": _first_present(df, ART_MAP["section"]),
        "subsection": _first_present(df, ART_MAP["subsection"]),
        "headline": _first_present(df, ART_MAP["headline"]),
        "abstract": _first_present(df, ART_MAP["abstract"]),
        "news_desk": _first_present(df, ART_MAP["news_desk"]),
        "type_of_material": _first_present(df, ART_MAP["type_of_material"]),
        "keywords": _first_present(df, ART_MAP["keywords"]),
    })
    return out

def _normalize_comments_small(df: pd.DataFrame) -> pd.DataFrame:
    def _first_present_local(cands, default=pd.NA):
        for c in cands:
            if c in df.columns:
                return df[c]
        return pd.Series([default]*len(df))

    out = pd.DataFrame({
        "comment_id": _first_present_local(COM_MAP["comment_id"]),
        "parent_id": _first_present_local(COM_MAP["parent_id"]),
        "user": _first_present_local(COM_MAP["user"]),
        "comment": _first_present_local(COM_MAP["comment"]),
        "recommendations": _first_present_local(COM_MAP["recommendations"]),
        "reply_count": _first_present_local(COM_MAP["reply_count"]),
        "editors_selection": _first_present_local(COM_MAP["editors_selection"]),
        "date": _first_present_local(COM_MAP["date"]),
        "article_url": _first_present_local(COM_MAP["article_url"]).map(_clean_url),
        "article_id": _first_present_local(COM_MAP["article_id"]),
    })
    return out

# ------------------- Loader functions -------------------
def _find(paths_patterns: List[str]) -> List[str]:
    s = set()
    for pat in paths_patterns:
        s.update(glob.glob(pat))
    return sorted(s)

def load_articles_build_maps(dirpath: str):
    a_paths = _find([
        os.path.join(dirpath, "Articles*.csv"),
        os.path.join(dirpath, "*articles*.csv"),
        os.path.join(dirpath, "articles*.csv"),
        os.path.join(dirpath, "nyt-articles-2020.csv"),
    ])
    if not a_paths:
        raise FileNotFoundError(f"No article CSVs in {dirpath}")

    arts = pd.concat([pd.read_csv(p, low_memory=False) for p in a_paths], ignore_index=True)
    arts = _normalize_articles_small(arts)

    # Deduplicate
    arts = arts.sort_values("pub_date").drop_duplicates(subset=["article_id","url"], keep="first")

    cols = ["headline","abstract","pub_date","section","subsection","news_desk","type_of_material","keywords","url"]

    by_id: Dict[object, Tuple] = {}
    for aid, t in zip(arts["article_id"], arts[cols].itertuples(index=False, name=None)):
        if pd.isna(aid): continue
        by_id[aid] = t
    for aid, t in zip(arts["article_id"], arts[cols].itertuples(index=False, name=None)):
        if pd.isna(aid): continue
        by_id[str(aid)] = t

    by_url: Dict[str, Tuple] = {}
    for u, t in zip(arts["url"], arts[cols].itertuples(index=False, name=None)):
        if pd.isna(u): continue
        by_url[str(u)] = t

    return by_id, by_url

def comment_files(dirpath: str) -> List[str]:
    c_paths = _find([
        os.path.join(dirpath, "Comments*.csv"),
        os.path.join(dirpath, "*comments*.csv"),
        os.path.join(dirpath, "comments*.csv"),
        os.path.join(dirpath, "nyt-comments-2020*.csv"),
    ])
    if not c_paths:
        raise FileNotFoundError(f"No comment CSVs in {dirpath}")
    return c_paths

# ------------------- Streaming join & election filtering -------------------
def process_comment_stream(by_id, by_url, cpaths: List[str], out_filtered_csv: str, chunksize: int = 25_000):
    os.makedirs(os.path.dirname(out_filtered_csv), exist_ok=True)
    if os.path.exists(out_filtered_csv):
        os.remove(out_filtered_csv)

    for p in cpaths:
        print(f"Processing: {os.path.basename(p)}")
        for i, raw in enumerate(pd.read_csv(p, low_memory=False, chunksize=chunksize)):
            c = _normalize_comments_small(raw)

            cols = ["headline","abstract","pub_date","section","subsection","news_desk","type_of_material","keywords","url"]
            for col in cols:
                c[col] = pd.NA

            # --- ID join (raw, then str) ---
            aid_raw = c["article_id"]
            got_raw = aid_raw.map(by_id)
            mask_raw = got_raw.notna()
            id_hits_raw = int(mask_raw.sum())
            if id_hits_raw:
                vals = got_raw[mask_raw].tolist()
                filled = pd.DataFrame(vals, index=c.index[mask_raw], columns=cols)
                for col in cols:
                    c.loc[mask_raw, col] = filled[col].values

            mask_need_id = c["headline"].isna()
            got_str = aid_raw.astype(str).where(mask_need_id, None).map(by_id)
            mask_str = got_str.notna()
            id_hits_str = int(mask_str.sum())
            if id_hits_str:
                vals2 = got_str[mask_str].tolist()
                filled2 = pd.DataFrame(vals2, index=c.index[mask_str], columns=cols)
                for col in cols:
                    c.loc[mask_str, col] = filled2[col].values

            # --- URL fallback ---
            mask_need_url = c["headline"].isna()
            url_clean = c.loc[mask_need_url, "article_url"].map(_clean_url)
            got_url = url_clean.map(by_url)
            mask_url = got_url.notna()
            url_hits = int(mask_url.sum())
            if url_hits:
                vals3 = got_url[mask_url].tolist()
                filled3 = pd.DataFrame(vals3, index=c.index[mask_need_url][mask_url], columns=cols)
                for col in cols:
                    c.loc[c.index[mask_need_url][mask_url], col] = filled3[col].values

            # --- Election-only filter ---
            keep_mask = c.apply(
                lambda r: _is_us_election_row(
                    r["section"], r["subsection"], r["headline"], r["abstract"], r["keywords"],
                    url_hint=r.get("url"), url_hint2=r.get("article_url")
                ),
                axis=1
            )
            out = c.loc[keep_mask, OUT_COLS].copy()

            if len(out):
                out.to_csv(out_filtered_csv, mode="a", header=not os.path.exists(out_filtered_csv), index=False)

            print(f"  chunk {i+1}: total={len(c):,}, id_hits={id_hits_raw + id_hits_str:,}, url_hits={url_hits:,}, kept={len(out):,}")

            del raw, c, out, got_raw, got_str, got_url
            gc.collect()

# ------------------- Master runner -------------------
def build_dataset_ultra(path_2017_2018: str, path_2020: str, out_dir="/content/nyt_outputs", chunksize=25_000):
    os.makedirs(out_dir, exist_ok=True)

    # 2017/2018
    by_id_A, by_url_A = load_articles_build_maps(path_2017_2018)
    filtA = os.path.join(out_dir, "us_elections_2017_2018.csv")
    process_comment_stream(by_id_A, by_url_A, comment_files(path_2017_2018), filtA, chunksize=chunksize)

    # 2020
    by_id_B, by_url_B = load_articles_build_maps(path_2020)
    filtB = os.path.join(out_dir, "us_elections_2020.csv")
    process_comment_stream(by_id_B, by_url_B, comment_files(path_2020), filtB, chunksize=chunksize)

    # combine
    combo = os.path.join(out_dir, "us_elections_combined.csv")
    if os.path.exists(combo): os.remove(combo)
    for f in [filtA, filtB]:
        if os.path.exists(f):
            pd.read_csv(f, low_memory=False).to_csv(combo, mode="a", header=not os.path.exists(combo), index=False)

    print("Done. Outputs in:", out_dir)


In [4]:
import kagglehub

path_17_18 = kagglehub.dataset_download("aashita/nyt-comments")
path_2020  = kagglehub.dataset_download("benjaminawd/new-york-times-articles-comments-2020")

build_dataset_ultra(
    path_2017_2018=path_17_18,
    path_2020=path_2020,
    out_dir="/content/nyt_outputs",
    chunksize=25_000   # try 10_000 if your runtime RAM is low
)


Using Colab cache for faster access to the 'nyt-comments' dataset.
Using Colab cache for faster access to the 'new-york-times-articles-comments-2020' dataset.
Processing: CommentsApril2017.csv
  chunk 1: total=25,000, id_hits=25,000, url_hits=0, kept=4,244
  chunk 2: total=50,000, id_hits=25,000, url_hits=0, kept=6,716
  chunk 3: total=50,000, id_hits=25,000, url_hits=0, kept=2,114
  chunk 4: total=50,000, id_hits=25,000, url_hits=0, kept=1,513
  chunk 5: total=50,000, id_hits=25,000, url_hits=0, kept=2,671
  chunk 6: total=50,000, id_hits=25,000, url_hits=0, kept=2,140
  chunk 7: total=50,000, id_hits=25,000, url_hits=0, kept=5,981
  chunk 8: total=50,000, id_hits=25,000, url_hits=0, kept=749
  chunk 9: total=50,000, id_hits=25,000, url_hits=0, kept=3,639
  chunk 10: total=37,664, id_hits=18,832, url_hits=0, kept=4,657
Processing: CommentsApril2018.csv
  chunk 1: total=25,000, id_hits=25,000, url_hits=0, kept=8,532
  chunk 2: total=50,000, id_hits=25,000, url_hits=0, kept=8,177
  chun

KeyboardInterrupt: 

In [5]:
!wc -l /content/nyt_outputs/us_elections_2017_2018.csv
!wc -l /content/nyt_outputs/us_elections_2020.csv
!wc -l /content/nyt_outputs/us_elections_combined.csv
!tail -n 3 /content/nyt_outputs/us_elections_2020.csv


402487 /content/nyt_outputs/us_elections_2017_2018.csv
9898901 /content/nyt_outputs/us_elections_2020.csv
10098834 /content/nyt_outputs/us_elections_combined.csv
Fold itself in praying the Provider
For Peace Food Love and Harmony",0.0,0.0,True,2021-01-04 02:55:30,nyt://interactive/61cd3792-6672-5c18-9377-6cb118b4a125,,,2020-12-31 10:01:02+00:00,The Upshot,,How We Got Through and What We Missed Most: Lessons From a Pandemic Year,"Readers across the country told us about their lockdown life. Baking, bourbon and biting nails? Check. But also, resilience and hope.",,,"['Quarantine (Life and Culture)', 'Coronavirus (2019-nCoV)', 'Polls and Public Opinion', 'Anxiety and Stress']"
110887111.0,,Nancy,"I learned that the two of us only use three rolls of toilet paper a month, and if i go along with the panic and buy 24 rolls that shred when wet, i will be regretting it for a while.",1.0,0.0,False,2021-01-04 18:15:52,nyt://interactive/61cd3792-6672-5c18-9377-6cb118b4a125,,,2020-12-31 10:01:02+00

In [6]:
import pandas as pd, re, math, os

in_dir  = "/content/nyt_outputs"
files_in = [
    os.path.join(in_dir, "us_elections_2017_2018.csv"),
    os.path.join(in_dir, "us_elections_2020.csv"),
    os.path.join(in_dir, "us_elections_combined.csv"),
]

def _to_lower_str(v):
    if v is None: return ""
    try:
        if v is pd.NA or (isinstance(v, float) and math.isnan(v)): return ""
    except Exception:
        pass
    if pd.isna(v): return ""
    return str(v).strip().lower()

# Strict, election-only signals (no generic “polls” on its own)
CORE_ELECTION = re.compile(r"""
    \b(election|primary|caucus|runoff|midterm|ballot|absentee|mail[- ]in|
       early\ voting|voter|voting|turnout|campaign|candidate|fec|debate|endorsement)s?\b
""", re.IGNORECASE | re.VERBOSE)

URL_ELECTION_SEGMENTS = (
    "/elections/", "/politics/elections", "/us/politics/elections",
    "/news-event/2020-election", "/news-event/2022-midterms", "/news-event/2024-election"
)

SECTIONS_ALLOWED = {"politics"}  # (You can add "U.S." if you want broader scope)
SUBSTR_ALLOWED   = ("election","campaign","midterm","primary","vote")

# Exclude clearly non-election pandemic pieces *unless* they also match CORE_ELECTION
EXCLUDE_PANDEMIC = re.compile(r"\b(covid|coronavirus|pandemic|quarantine|lockdown)\b", re.IGNORECASE)

def looks_like_election_row(row):
    section    = _to_lower_str(row.get("section"))
    subsection = _to_lower_str(row.get("subsection"))
    headline   = _to_lower_str(row.get("headline"))
    abstract   = _to_lower_str(row.get("abstract"))
    keywords   = row.get("keywords")
    url1       = _to_lower_str(row.get("url"))
    url2       = _to_lower_str(row.get("article_url"))

    # Section/subsection cues
    if section in SECTIONS_ALLOWED: return True
    if any(k in subsection for k in SUBSTR_ALLOWED): return True

    # URL cues (explicit election paths)
    if any(seg in url1 for seg in URL_ELECTION_SEGMENTS): return True
    if any(seg in url2 for seg in URL_ELECTION_SEGMENTS): return True

    # Keywords / text cues
    kwtxt = ""
    if isinstance(keywords, list):
        kwtxt = " ".join(_to_lower_str(k) for k in keywords if k is not None)
    else:
        kwtxt = _to_lower_str(keywords)

    strong_text_hit = bool(
        CORE_ELECTION.search(kwtxt) or CORE_ELECTION.search(headline) or CORE_ELECTION.search(abstract)
    )

    # If pandemic-y but no strong election terms → exclude
    if EXCLUDE_PANDEMIC.search(" ".join([headline, abstract, kwtxt])) and not strong_text_hit:
        return False

    return strong_text_hit

def refilter_file(path_in):
    path_out = path_in.replace(".csv", "_strict.csv")
    if os.path.exists(path_out):
        os.remove(path_out)

    kept = 0
    total = 0
    for chunk in pd.read_csv(path_in, low_memory=False, chunksize=200_000):
        mask = chunk.apply(looks_like_election_row, axis=1)
        out  = chunk.loc[mask]
        total += len(chunk)
        kept  += len(out)
        out.to_csv(path_out, mode="a", header=not os.path.exists(path_out), index=False)
        print(f"{os.path.basename(path_in)}: processed={total:,}, kept={kept:,}")
    return path_out, kept, total

strict_paths = []
for f in files_in:
    if os.path.exists(f):
        strict_paths.append(refilter_file(f)[0])

print("\nStrict files written:")
for p in strict_paths:
    print(" -", p)


us_elections_2017_2018.csv: processed=200,000, kept=194,662
us_elections_2017_2018.csv: processed=400,000, kept=387,492
us_elections_2017_2018.csv: processed=402,486, kept=389,978


KeyboardInterrupt: 

In [7]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/nyt_outputs
!cp -f /content/nyt_outputs/*.csv /content/drive/MyDrive/nyt_outputs/


Mounted at /content/drive
^C


In [8]:
!cp -f /content/nyt_outputs/us_elections_2017_2018.csv /content/drive/MyDrive/nyt_outputs/
!cp -f /content/nyt_outputs/us_elections_combined.csv /content/drive/MyDrive/nyt_outputs/


In [9]:
!ls -lh /content/drive/MyDrive/nyt_outputs


total 4.9G
-rw------- 1 root root 1006M Oct 21 18:47 domestic_politics_econ_2017_2018.csv
-rw------- 1 root root  306M Oct 21 18:49 us_elections_2017_2018.csv
-rw------- 1 root root  297M Oct 21 18:47 us_elections_2017_2018_strict.csv
-rw------- 1 root root  3.2G Oct 21 18:47 us_elections_2020.csv
-rw------- 1 root root  121M Oct 21 18:47 us_elections_2020_strict.csv


In [10]:
!cp -f /content/nyt_outputs/us_elections_combined.csv /content/drive/MyDrive/nyt_outputs/
